In [ ]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    # GPU를 디바이스로 설정
    device = torch.device("cuda")
else:
    # CPU를 디바이스로 설정
    device = torch.device("cpu")

In [ ]:
!nvidia-smi

Wed May 24 08:17:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
! ps -ef | grep python

root          75       7  2 08:13 ?        00:00:06 [python3] <defunct>
root          76       7  0 08:13 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root         117       7  1 08:13 ?        00:00:03 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
root         732     117  3 08:15 ?        00:00:06 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-f63deb15-8e4c-43cf-82c9-5f389c9731d8.json
root         781       1  0 08:15 ?        00:00:00 /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/debugpy/adapter --for-server 42755 --host 127.0.0.1 --port 21150 --server-access-token 82482759241b6c6042f8bef51af8e46c95b9919683cf7308148a931fd236cd37
root        1053       1  0 08:15 ?        00:00:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/

In [ ]:
!pip install -qq  accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 19.7 MB/s eta 0:00:00


In [ ]:
!pip install -qq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)
_ = model.eval()

prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 해결할 수 있을 것"이라고 강조했다.[EOS]


In [ ]:
prompt = '''애니메이션에서나 볼 수 있었던 자동차 변신 로봇이 일본에서 실제로 만들어진다.
11일 소프트뱅크 계열 로봇 제조업체인 아스라테크(Asratec)와 산세이 테크놀로지, 브레이브 로보틱스가 3사 합작으로 변신 로봇 '제이 다이트 라이드(J-deite RIDE)'를 제작한다고 발표했다..
이 로봇은 아스라테크와 브레이브 로보틱스가 참여한 로봇 제조 계획 "프로젝트 제이 다이트'의 일환으로 지난해 6월부터 개발이 진행되고 있다 하드웨어 설계 ·제작은 브레이브 로보틱스가 맡고 제어용 소프트웨어 '브이시도(V-Sido)'의 개발은 아스라테크가 담당한다.
산세이 테크놀로지는 로봇의 양산화를 위해서 합류했다 '제이 다이트 라이드'는 내년에 완성돼 테마 파크와 각종 이벤트 참여, 기업 홍보 등의 목적으로 쓰이게 된다. 
이후에는 기체의 크기를 키워 2020년에는 5m에 달하는 '제이 다이트 오리지널'로 재탄생할 예정이다.
디자인은 애니메이션 '기동전사 건담', '달려라 번개호' 등 유명 애니메이션에 등장하는 로봇을 창조한 오오카와라 쿠니오가 맡았다 오오카와라는 '제이 다이트 라이드' 외에 프로젝트에서 제작할 예정인 다른 로봇의 디자인도 맡는다.
'제이다이트 라이드'의 머리 부분 스케치 언뜻 보기에도 건담이 연상된다.
이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다 로봇 모드에서는 두 다리로 걸을 수 있다. 
차량 모드에서는 평범한 자동차와 마찬가지로 사람이 운전석에 앉아 조작하면 된다. 
'철인28호'처럼 로봇과 떨어져 원격 조종을 해도 된다.
최대 2명이 탈 수 있다. 
로봇이 두 다리로 걷는 속도는 시속 5km에 불과하지만 발 부분의 바퀴를 이용하면 한시간에 최고 30km를 갈 수 있다. 
차로 변신했을 때 최고 속도는 시속 60km이다. 
한편 '제이다이트 라이드'의 1.5m짜리 시제품은 14일(현지시간) 미국 올랜도에서 열리는 '국제 놀이기구 엑스포(IAAPA Attractions Expo 2016)'에서 선보일 예정이다

한줄 요약: '''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.5, max_length=512)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

애니메이션에서나 볼 수 있었던 자동차 변신 로봇이 일본에서 실제로 만들어진다.
11일 소프트뱅크 계열 로봇 제조업체인 아스라테크(Asratec)와 산세이 테크놀로지, 브레이브 로보틱스가 3사 합작으로 변신 로봇 '제이 다이트 라이드(J-deite RIDE)'를 제작한다고 발표했다..
이 로봇은 아스라테크와 브레이브 로보틱스가 참여한 로봇 제조 계획 "프로젝트 제이 다이트'의 일환으로 지난해 6월부터 개발이 진행되고 있다 하드웨어 설계 ·제작은 브레이브 로보틱스가 맡고 제어용 소프트웨어 '브이시도(V-Sido)'의 개발은 아스라테크가 담당한다.
산세이 테크놀로지는 로봇의 양산화를 위해서 합류했다 '제이 다이트 라이드'는 내년에 완성돼 테마 파크와 각종 이벤트 참여, 기업 홍보 등의 목적으로 쓰이게 된다. 
이후에는 기체의 크기를 키워 2020년에는 5m에 달하는 '제이 다이트 오리지널'로 재탄생할 예정이다.
디자인은 애니메이션 '기동전사 건담', '달려라 번개호' 등 유명 애니메이션에 등장하는 로봇을 창조한 오오카와라 쿠니오가 맡았다 오오카와라는 '제이 다이트 라이드' 외에 프로젝트에서 제작할 예정인 다른 로봇의 디자인도 맡는다.
'제이다이트 라이드'의 머리 부분 스케치 언뜻 보기에도 건담이 연상된다.
이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다 로봇 모드에서는 두 다리로 걸을 수 있다. 
차량 모드에서는 평범한 자동차와 마찬가지로 사람이 운전석에 앉아 조작하면 된다. 
'철인28호'처럼 로봇과 떨어져 원격 조종을 해도 된다.
최대 2명이 탈 수 있다. 
로봇이 두 다리로 걷는 속도는 시속 5km에 불과하지만 발 부분의 바퀴를 이용하면 한시간에 최고 30km를 갈 수 있다. 
차로 변신했을 때 최고 속도는 시속 60km이다. 
한편 '제이다이트 라이드'의 1.5m짜리 시제품은 14일(현지시간) 미국 올랜도에서 열리는 '국제 놀이기구 엑스포(IAAPA Attractions Expo 2016)'에서 선보일 예정이다

한줄 요약:  11일